# 参考サイト
- [FSRCNN_withPytorch](https://github.com/yjn870/FSRCNN-pytorch)

- [個人的にわかりやすいpytorchチュートリアル](https://qiita.com/mckeeeen/items/e255b4ac1efba88d0ca1)

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data.dataloader import DataLoader

from collections import OrderedDict
import os

# モデル定義

In [13]:
class FSRCNN(nn.Module):
    def __init__(self,scale_factor):
        super(FSRCNN, self).__init__()
        self.first_layer = nn.Sequential(OrderedDict([
            ("fl_conv",nn.Conv2d(1,56,kernel_size=5,stride=1,padding=0)),
            ("fl_PReLU",nn.PReLU())
        ]))
        
        self.middle_layer =nn.Sequential(OrderedDict([
            ("ml_conv1",nn.Conv2d(56,12,1,1,0)),
            ("ml_PReLU1",nn.PReLU(56)),
            
            ("ml_conv2",nn.Conv2d(12,12,3,1,1)),
            ("ml_PReLU2",nn.PReLU(12)),
            
            ("ml_conv3",nn.Conv2d(12,12,3,1,1)),
            ("ml_PReLU3",nn.PReLU(12)),
            
            ("ml_conv4",nn.Conv2d(12,12,3,1,1)),
            ("ml_PReLU4",nn.PReLU(12)),
            
            ("ml_conv5",nn.Conv2d(12,12,3,1,1)),
            ("ml_PReLU5",nn.PReLU(12)),
            
            ("ml_conv6",nn.Conv2d(12,56,1,1,1)),
            ("ml_PReLU6",nn.PReLU(56)),
        ]))
        
        self.last_layer = nn.Sequential(OrderedDict([
            ("ll_Deconv",nn.ConvTranspose2d(56,1,
                                            kernel_size=9,stride=scale_factor,padding=4))
        ]))
        
        nn.init.normal_(self.first_layer.fl_conv.weight, 0.0, 0.0378)
        nn.init.zeros_(self.first_layer.fl_conv.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv1.weight, 0.0, 0.3536)
        nn.init.zeros_(self.middle_layer.ml_conv1.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv2.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv2.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv3.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv3.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv4.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv4.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv5.weight, 0.0, 0.1179)
        nn.init.zeros_(self.middle_layer.ml_conv5.bias.data)
        nn.init.normal_(self.middle_layer.ml_conv6.weight, 0.0, 0.189)
        nn.init.zeros_(self.middle_layer.ml_conv6.bias.data)
        nn.init.normal_(self.last_layer.ll_Deconv.weight, 0.0, 0.0001)
        nn.init.zeros_(self.last_layer.ll_Deconv.bias.data)
        
    def forward(self,x):
        x = self.first_layer(x)
        x = self.middle_layer(x)
        x = self.last_layer(x)
        return x

# データセットの作成

In [36]:
import h5py
import numpy as np
from torch.utils.data import Dataset

In [53]:
class TrainDataset(Dataset):
    def __init__(self, h5_file):
        super(TrainDataset, self).__init__()
        self.h5_file = h5_file
        
    def __getitem__(self, idx):
        with h5py.File(self.h5_file, 'r') as f:
            return np.expand_dims(f['lr'][idx] / 255.,0), \
                   np.expand_dims(f['hr'][idx] /255.,0)
    
    def __len__(self):
        with h5py.File(self.h5_file, 'r') as f:
            return len(f['lr'])

#### [np.expand_dims()とは](https://teratail.com/questions/146318)
- np.expand_dims() は、第2引数の axis で指定した場所の直前に dim=1 を挿入します。負の値の場合は、Python の添字記法と同じ末尾からの参照になります。

In [51]:
img = np.zeros((100, 100, 3), dtype=float)
print(img.shape)  # (100, 100, 3)
_img = np.expand_dims(img,axis=0)#0番目の軸の前にdim=1を挿入
print(_img.shape)
print(255.)

(100, 100, 3)
(1, 100, 100, 3)
255.0


# main

### 1. arg系で指定されていたものの書き換え

[num_workerとは](https://stackoverflow.com/questions/53998282/how-does-the-number-of-workers-parameter-in-pytorch-dataloader-actually-work)
- num_workers = 2の場合、最大2人のワーカーが同時にデータをRAMに入れます。

In [16]:
train_file = ""#訓練所のファイル
eval_file = "" #eval=評価するためのファイル
outputs_dir = "output" #出力ファイル
weights_file = "" #重みファイル
scale = 2 #画像の拡大率　default=2
lr = 1e-3  #学習率 1e-3
batch_size=16 #バッチサイズ 16
num_epochs=20 #エポック数
num_workers= 8 #num_workersでいくつのコアでデータをロードするか指定(デフォルトはメインのみ)
seed = 123

### 2. 出力先ディレクトリの作成と指定
- 後で実行

In [22]:
scale_output_dir = os.path.join(outputs_dir,'x{}'.format(scale))
print(scale_output_dir)
#if not os.path.join(scale_output_dir):
#    os.makedirs(scale_output_dir)

### 3. cuDNNのベンチマークモードをオンにするかどうかのオプション
- Trueにするとオートチューナーがネットワークの構成に対し最適なアルゴリズムを見つけるため、高速化されます.
- [benchmark=Trueとは](https://qiita.com/koshian2/items/9877ed4fb3716eac0c37)

In [27]:
cudnn.benchmark = True

### 4. 使うデバイスを指定

In [28]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


### 5. [シード値の初期化](https://qrunch.net/@haru256/entries/HW8uMhxBnEJ1qnFr)
- torchでのRNGを初期化
- RNG(Random Number Generator)

In [29]:
torch.manual_seed(seed)

### 6. 作成したモデル(NN)のインスタンスを生成

In [31]:
model = FSRCNN(scale_factor=scale).to(device)

### 7. 損失関数(平均二乗誤差)のインスタンスを生成

In [32]:
criterion = nn.MSELoss()

### 8.Adamのインスタンスを生成
- 要素が辞書型のlist型のデータでパラメータを設定している
- Variableの代わりに辞書型で返すことができる
- 詳しくはこのサイト→　https://pytorch.org/docs/stable/optim.html

In [34]:
optimizer = optim.Adam([
    {'params': model.first_layer.parameters()},
    {'params': model.middle_layer.parameters()},
    {'params': model.last_layer.parameters(), 'lr': lr *0.1}
], lr = lr)

### 9. [データのロード](https://qiita.com/takurooo/items/e4c91c5d78059f92e76d)

In [54]:
train_dataset = TrainDataset(train_file)
train_dataloader = DataLoader()